# Walkthrough of iXnos makefile

In [1]:
import os

working_dir = "../iXnos"
os.chdir(working_dir)

In [3]:
# download the Iwasaki data from SRA
!fastq-dump SRR2075925 -O expts/iwasaki/process
!fastq-dump SRR2075926 -O expts/iwasaki/process
# !fastq-dump SRR2075925 -O expts/iwasaki/process
# !fastq-dump SRR2075926 -O expts/iwasaki/process

Read 45615984 spots for SRR2075925
Written 45615984 spots for SRR2075925
Read 34606362 spots for SRR2075926
Written 34606362 spots for SRR2075926


In [2]:
# process the data
!bash expts/iwasaki/process/iwasaki.sh genome_data expts/iwasaki/process

genome_data
I'm making a human index!!!
rsem-synthesis-reference-transcripts genome_data/human.transcripts.13cds10 0 0 genome_data/gencode.v22.transcript.13cds10.fa
Transcript Information File is generated!
Group File is generated!
Extracted Sequences File is generated!

rsem-preref genome_data/human.transcripts.13cds10.transcripts.fa 1 genome_data/human.transcripts.13cds10
Refs.makeRefs finished!
Refs.saveRefs finished!
genome_data/human.transcripts.13cds10.idx.fa is generated!
genome_data/human.transcripts.13cds10.n2g.idx.fa is generated!

rsem-parse-alignments genome_data/human.transcripts.13cds10 expts/iwasaki/process/iwasaki.temp/iwasaki expts/iwasaki/process/iwasaki.stat/iwasaki expts/iwasaki/process/iwasaki.footprints.sam 1 -tag XM
Parsed 1000000 entries
Parsed 2000000 entries
Parsed 3000000 entries
Parsed 4000000 entries
Parsed 5000000 entries
Parsed 6000000 entries
Parsed 7000000 entries
Parsed 8000000 entries
Parsed 9000000 entries
Parsed 10000000 entries
Parsed 11000000 entr

In [ ]:
# Need to run separately in terminal
!python reproduce_scripts/process_data.py edit_sam_file iwasaki \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.sam


In [ ]:
!mkdir expts/iwasaki/plots

In [ ]:
# (Need to run separately in terminal)
# look at the size and frame of the ribosome footprints to figure out what 
# A-site offsets are appropriate
!python reproduce_scripts/process_data.py size_and_frame_analysis iwasaki \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam \
	genome_data/gencode.v22.transcript.13cds10.lengths.txt


In [ ]:
# (Need to run in separate terminal)
!python reproduce_scripts/process_data.py process_sam_file iwasaki \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam \
	genome_data/gencode.v22.transcript.13cds10.lengths.txt genome_data/gencode.v22.transcript.13cds10.fa

In [ ]:
!mkdir expts/iwasaki/lasagne_nn

# Running the models

## -5 to +4

In [ ]:
# this model goes from codons -5 to +4, plus the corresponding nucleotides. for the paper we ran this 10 times to get replicates, so thsi one is called rep 0, but we don't need to do that
# I don't remember what 70 and 32 are for, maybe the epochs we use later?
# 70 = num_epochs
# 32 = lr_decay

# model_name = cod_n5p4_nt_n15p14
# model_rep = 0
# expt_dir = expts/iwasaki 
# sam_fname = expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam
# gene_len_fname = genome_data/gencode.v22.transcript.13cds10.lengths.txt
# gene_seq_fname = genome_data/gencode.v22.transcript.13cds10.fa
# tr_codons_fname = expts/iwasaki/process/tr_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt
# te_codons_fname = expts/iwasaki/process/te_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt
# outputs_fname = expts/iwasaki/process/outputs.size.27.30.txt
# num_epochs = 70
# lr_decay = 32

!python reproduce_scripts/feat_neighborhood_nn_series.py \
	cod_n5p4_nt_n15p14 0 \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam \
	genome_data/gencode.v22.transcript.13cds10.lengths.txt genome_data/gencode.v22.transcript.13cds10.fa \
	expts/iwasaki/process/tr_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt expts/iwasaki/process/te_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt \
	expts/iwasaki/process/outputs.size.27.30.txt 70 \
	32

## -7 to +5

In [ ]:
# same but codons -7 to +5
!python reproduce_scripts/feat_neighborhood_nn_series.py \
	cod_n7p5_nt_n21p17 0 \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam \
	genome_data/gencode.v22.transcript.13cds10.lengths.txt genome_data/gencode.v22.transcript.13cds10.fa \
	expts/iwasaki/process/tr_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt expts/iwasaki/process/te_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt \
	expts/iwasaki/process/outputs.size.27.30.txt 70 \
	32

## Leave-one-out

In [ ]:
# an example of one of the leave-one-out runs: this is leaving out codon -7. in the full makefile we generate one of these commands for each positionecho
# HAVE NOT RUN YET
!python reproduce_scripts/leaveout_series.py \
	nocod-7_cod_n7p5_nt_n21p17 0 \
	expts/iwasaki expts/iwasaki/process/iwasaki.transcript.mapped.wts.sam \
	genome_data/gencode.v22.transcript.13cds10.lengths.txt genome_data/gencode.v22.transcript.13cds10.fa \
	expts/iwasaki/process/tr_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt expts/iwasaki/process/te_set_bounds.size.27.30.trunc.20.20.min_cts.200.min_cod.100.top.500.txt \
	expts/iwasaki/process/outputs.size.27.30.txt \
	70 \
	32

# Summarizing results etc.

In [ ]:
# save some stuff, make some diagnostic figures
!mkdir results/iwasaki
!mkdir results/iwasaki/full_cod_n5p4_nt_n15p14_rep0
!mkdir results/iwasaki/full_cod_n5p4_nt_n15p14_rep0/epoch70
!python reproduce_scripts/codon_scores.py \
	expts/iwasaki/lasagne_nn/full_cod_n5p4_nt_n15p14_rep0 \
	70


In [30]:
!cp expts/iwasaki/lasagne_nn/full_cod_n5p4_nt_n15p14_rep0/epoch70/codon_scores.tsv \
	results/iwasaki/full_cod_n5p4_nt_n15p14_rep0/epoch70/codon_scores.tsv
!cp expts/iwasaki/lasagne_nn/full_cod_n5p4_nt_n15p14_rep0/epoch70/codon_scores_colormap.pdf \
	results/iwasaki/full_cod_n5p4_nt_n15p14_rep0/epoch70/codon_scores_colormap.pdf

In [ ]:
# The 1 is how many replicates were run; in this case only 1
!mkdir results/iwasaki/feat_neighborhood_series
!python reproduce_scripts/aggregate_corrs.py pearson \
	expts/iwasaki/lasagne_nn 70 \
	1 \ 
	results/iwasaki/feat_neighborhood_series/feat_neighborhood_corrs.txt \
	full_cod_n5p4_nt_n15p14 full_cod_n7p5_nt_n21p17
!mkdir results/iwasaki/leaveout_series

In [ ]:
# this needs all the leave-one-out runs from -7 to +5
!python reproduce_scripts/aggregate_corrs.py pearson \
	expts/iwasaki/lasagne_nn 70 \
	10 \
	results/iwasaki/leaveout_series/leaveout_corrs.txt \
	nocod-7_cod_n7p5_nt_n21p17 nocod-6_cod_n7p5_nt_n21p17 nocod-5_cod_n7p5_nt_n21p17 nocod-4_cod_n7p5_nt_n21p17 nocod-3_cod_n7p5_nt_n21p17 nocod-2_cod_n7p5_nt_n21p17 nocod-1_cod_n7p5_nt_n21p17 nocod0_cod_n7p5_nt_n21p17 nocod1_cod_n7p5_nt_n21p17 nocod2_cod_n7p5_nt_n21p17 nocod3_cod_n7p5_nt_n21p17 nocod4_cod_n7p5_nt_n21p17 nocod5_cod_n7p5_nt_n21p17


In [ ]:
!python reproduce_scripts/plot_nn.py \
	expts/iwasaki full_cod_n5p4_nt_n15p14_rep0 \
	70
!cp -r expts/iwasaki/lasagne_nn/full_cod_n5p4_nt_n15p14_rep0/plots \
	results/iwasaki/full_cod_n5p4_nt_n15p14_rep0
